In [1]:
%pip install tensorflow

  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 3.7/390.3 MB 54.1 MB/s eta 0:00:08
   -- ------------------------------------- 19.9/390.3 MB 54.6 MB/s eta 0:00:07
   --- ------------------------------------ 31.5/390.3 MB 55.4 MB/s eta 0:00:07
   ---- ----------------------------------- 41.9/390.3 MB 54.4 MB/s eta 0:00:07
   ----- ---------------------------------- 54.8/390.3 MB 56.3 MB/s eta 0:00:06
   ------ --------------------------------- 67.1/390.3 MB 57.0 MB/s eta 0:00:06
   -------- ------------------------------- 81.3/390.3 MB 58.2 MB/s eta 0:00:06
   --------- ------------------------------ 94.4/390.3 MB 59.0 MB/s eta 0:00:06
   ---------- ---------------------------- 107.5/390.3 MB 59.7 MB/s eta 0:00:05
   ------------ -------------------------- 121.1/390.3 MB 60.4 MB/s eta 0:00:05
   ------------- ------------------------- 133.2/390.3 MB 60.3 MB/s

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import glob

In [9]:
path = './data/'

In [ ]:
df_price = pd.read_csv(path + '甘藍初秋_台北一日交易行情.csv')
df_price.head()
###只需要用到民國104(2015)~113(2024)

,日期,市場,產品,上價,中價,下價,平均價,增減%,交易量(公斤),增減%.1
0,085/01/01,109 台北一,LA1 甘藍 初秋,12.0,9.0,6.0,8.8,-,"75,779",-
1,085/01/02,109 台北一,LA1 甘藍 初秋,13.0,10.0,7.0,11.2,27,"58,780",-22
2,085/01/03,109 台北一,LA1 甘藍 初秋,13.0,10.0,7.0,10.1,-10,"103,999",77
3,085/01/05,109 台北一,LA1 甘藍 初秋,11.0,9.0,7.0,8.1,-,"108,342",-
4,085/01/06,109 台北一,LA1 甘藍 初秋,10.0,8.0,6.0,7.9,-2,"113,928",5


In [ ]:
df_typhoon = pd.read_csv(path + 'typhoon_data.csv')
df_typhoon.head()
## 2015~2024

,年份,颱風編號,颱風名稱,颱風生命期間,颱風生命期間中心最低氣壓(hPa),颱風生命期間中心最大風速(m/s),颱風生命期間最大7級風暴風半徑(km),颱風生命期間最大10級風暴風半徑(km),警報發布報數
0,2024,202426,帕布(PABUK),2024-12-23 06:00~2024-12-25 00:00,1000,18,80,---,---
1,2024,202425,天兔(USAGI),2024-11-11 18:00~2024-11-16 03:00,945,45,150,50,19
2,2024,202424,萬宜(MAN-YI),2024-11-09 06:00~2024-11-20 00:00,900,60,180,90,---
3,2024,202423,桔梗(TORAJI),2024-11-09 06:00~2024-11-14 06:00,965,38,150,50,---
4,2024,202422,銀杏(YINXING),2024-11-03 18:00~2024-11-12 06:00,930,51,180,90,---


In [15]:
path = './data/daily_min_temperature/'
df_mintemp = pd.read_csv(path + '466920-2016-MinAirTemperature-day.csv')
df_mintemp.head()

,日/月,1,2,3,4,5,6,7,8,9,10,11,12
0,01,16.5 / 2016/01/01 01:31:00,11.7 / 2016/02/01 22:49:00,12.8 / 2016/03/01 00:16:00,18.2 / 2016/04/01 00:19:00,21.0 / 2016/05/01 05:31:00,28.8 / 2016/06/01 04:35:00,27.2 / 2016/07/01 04:31:00,27.2 / 2016/08/01 02:27:00,25.1 / 2016/09/01 05:47:00,27.2 / 2016/10/01 03:53:00,20.8 / 2016/11/01 18:04:00,18.7 / 2016/12/01 16:37:00
1,02,17.2 / 2016/01/02 05:50:00,11.5 / 2016/02/02 00:58:00,13.9 / 2016/03/02 04:53:00,19.2 / 2016/04/02 05:53:00,22.2 / 2016/05/02 03:26:00,25.9 / 2016/06/02 11:23:00,25.9 / 2016/07/02 15:29:00,28.1 / 2016/08/02 22:42:00,26.0 / 2016/09/02 18:19:00,26.2 / 2016/10/02 06:06:00,19.2 / 2016/11/02 05:16:00,18.8 / 2016/12/02 05:01:00
2,03,18.3 / 2016/01/03 03:52:00,12.7 / 2016/02/03 00:01:00,14.1 / 2016/03/03 05:15:00,21.1 / 2016/04/03 05:36:00,24.7 / 2016/05/03 04:15:00,25.9 / 2016/06/03 01:18:00,24.8 / 2016/07/03 17:31:00,27.0 / 2016/08/03 04:23:00,26.1 / 2016/09/03 04:09:00,25.6 / 2016/10/03 06:34:00,19.5 / 2016/11/03 03:51:00,19.0 / 2016/12/03 00:02:00
3,04,18.7 / 2016/01/04 01:04:00,14.6 / 2016/02/04 06:10:00,15.9 / 2016/03/04 06:26:00,21.4 / 2016/04/04 19:43:00,24.7 / 2016/05/04 02:41:00,27.6 / 2016/06/04 05:27:00,25.9 / 2016/07/04 04:43:00,27.3 / 2016/08/04 05:43:00,27.1 / 2016/09/04 05:58:00,25.5 / 2016/10/04 04:36:00,20.9 / 2016/11/04 09:23:00,21.2 / 2016/12/04 05:29:00
4,05,19.0 / 2016/01/05 02:57:00,13.4 / 2016/02/05 21:33:00,17.5 / 2016/03/05 05:46:00,21.4 / 2016/04/05 00:02:00,24.3 / 2016/05/05 18:29:00,24.6 / 2016/06/05 17:01:00,27.4 / 2016/07/05 05:40:00,27.9 / 2016/08/05 05:37:00,27.5 / 2016/09/05 01:30:00,26.0 / 2016/10/05 03:12:00,19.8 / 2016/11/05 03:20:00,21.2 / 2016/12/05 23:34:00


In [22]:
import pandas as pd
import os

# 定義檔案路徑
input_path = './data/daily_min_temperature/'
output_path = './data/cleaned_min_temperature/'

# 如果輸出目錄不存在，則創建它
if not os.path.exists(output_path):
    os.makedirs(output_path)

# 要處理的年份
years = range(2016, 2025)

# 批量處理每個檔案
for year in years:
    file_name = f'466920-{year}-MinAirTemperature-day.csv'
    input_file = os.path.join(input_path, file_name)

    # 讀取 CSV 檔案
    df = pd.read_csv(input_file)

    # 清理溫度欄位，只保留 `/` 前的數字
    df_cleaned = df.applymap(lambda x: x.split('/')[0] if isinstance(x, str) and '/' in x else x)

    # 輸出清理後的數據到新的 CSV 文件
    output_file = os.path.join(output_path, f'{file_name}')
    df_cleaned.to_csv(output_file, index=False)

    print(f"已處理並儲存：{output_file}")

print("所有檔案已成功處理！")


已處理並儲存：./data/cleaned_min_temperature/466920-2016-MinAirTemperature-day.csv
已處理並儲存：./data/cleaned_min_temperature/466920-2017-MinAirTemperature-day.csv
已處理並儲存：./data/cleaned_min_temperature/466920-2018-MinAirTemperature-day.csv
已處理並儲存：./data/cleaned_min_temperature/466920-2019-MinAirTemperature-day.csv
已處理並儲存：./data/cleaned_min_temperature/466920-2020-MinAirTemperature-day.csv
已處理並儲存：./data/cleaned_min_temperature/466920-2021-MinAirTemperature-day.csv
已處理並儲存：./data/cleaned_min_temperature/466920-2022-MinAirTemperature-day.csv
已處理並儲存：./data/cleaned_min_temperature/466920-2023-MinAirTemperature-day.csv
已處理並儲存：./data/cleaned_min_temperature/466920-2024-MinAirTemperature-day.csv
所有檔案已成功處理！


C:\Users\User\AppData\Local\Temp\ipykernel_21404\2865284357.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df.applymap(lambda x: x.split('/')[0] if isinstance(x, str) and '/' in x else x)
C:\Users\User\AppData\Local\Temp\ipykernel_21404\2865284357.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df.applymap(lambda x: x.split('/')[0] if isinstance(x, str) and '/' in x else x)
C:\Users\User\AppData\Local\Temp\ipykernel_21404\2865284357.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df.applymap(lambda x: x.split('/')[0] if isinstance(x, str) and '/' in x else x)
C:\Users\User\AppData\Local\Temp\ipykernel_21404\2865284357.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df.applymap(lambda x: x.split('/')[0] if isinstance(x, str) and '/' in x else x)
C:\Users\User\AppDat

In [ ]:
path = './data/cleaned_min_temperature/'
df_mintemp = pd.read_csv(path + '466920-2016-MinAirTemperature-day.csv')
df_mintemp.head()

In [21]:
path = './data/daily_precipitation/'
df_precipitation = pd.read_csv(path + '466920-2015-Precipitation-day.csv')
df_precipitation.head()

,日/月,1,2,3,4,5,6,7,8,9,10,11,12
0,01,0.0,0.0,2.0,0.0,2.4,0.2,0.0,0.0,24.2,0.0,1.2,0.0
1,02,0.0,0.0,T,0.0,1.3,0.0,0.0,0.0,0.0,58.8,T,0.0
2,03,0.0,3.5,T,0.0,1.1,0.0,0.0,0.0,T,0.0,0.7,0.0
3,04,0.0,3.0,1.0,0.0,39.6,0.9,1.3,0.0,T,0.0,1.0,0.2
4,05,0.0,4.0,1.5,0.0,41.3,0.0,0.0,17.2,0.0,0.0,0.0,T


預處理csv

訓練模型

In [2]:
# Step 1: 整合所有 CSV 文件
file_paths = glob.glob("data/train.csv")  # 修改為您的實際文件路徑模式
all_data = pd.DataFrame()

for file in file_paths:
    data = pd.read_csv(file)
    all_data = pd.concat([all_data, data], ignore_index=True)

print(f"數據集大小: {all_data.shape}")

# Step 2: 數據預處理
# 假設數據集包含 '日期' 和 '目標值' 列，您需要根據實際數據進行修改
all_data['日期'] = pd.to_datetime(all_data['日期'])  # 確保日期列格式正確

# 按日期排序
all_data = all_data.sort_values(by='日期')

# 填補缺失值
all_data = all_data.fillna(method='ffill').fillna(method='bfill')

# 特徵工程（這部分需要根據您的數據具體修改）
all_data['year'] = all_data['日期'].dt.year
all_data['month'] = all_data['日期'].dt.month
all_data['day'] = all_data['日期'].dt.day

# 歸一化目標值
scaler = MinMaxScaler(feature_range=(0, 1))
all_data['目標值'] = scaler.fit_transform(all_data[['目標值']])

# Step 3: 數據分割
# 篩選數據集
train_data = all_data[(all_data['year'] >= 2014) & (all_data['year'] <= 2020)]
val_data = all_data[(all_data['year'] == 2021) | (all_data['year'] == 2022)]
test_data = all_data[all_data['year'] == 2023]

# 創建時間序列數據
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length, :-1])
        y.append(data[i + sequence_length, -1])
    return np.array(X), np.array(y)

sequence_length = 30  # 使用過去 30 天的數據進行預測

# 訓練數據
train_values = train_data[['目標值']].values
X_train, y_train = create_sequences(train_values, sequence_length)

# 驗證數據
val_values = val_data[['目標值']].values
X_val, y_val = create_sequences(val_values, sequence_length)

# 測試數據
test_values = test_data[['目標值']].values
X_test, y_test = create_sequences(test_values, sequence_length)

# Step 4: 構建 LSTM 模型
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(50, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Step 5: 模型訓練
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1
)

# Step 6: 模型評估
y_test_pred = model.predict(X_test)

# 將數據反轉回原始尺度
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
y_test_pred = scaler.inverse_transform(y_test_pred)

mse = mean_squared_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)

print(f"Test MSE: {mse:.2f}")
print(f"Test R2: {r2:.2f}")

# Step 7: 保存模型
import joblib
joblib.dump(model, 'trained_lstm_model.h5')

數據集大小: (0, 0)


KeyError: '日期'

In [ ]:
# Step 8: 可視化結果
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(y_test, label='True Values')
plt.plot(y_test_pred, label='Predictions')
plt.legend()
plt.title('LSTM Predictions vs True Values')
plt.show()